In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.linalg.Vector

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val model = PipelineModel.load(modelPath)

model = pipeline_eafa8d640b8a


pipeline_eafa8d640b8a

In [8]:
def cutProbability (probabilityVector: Vector): Double = {
  probabilityVector(1)
}

val sparkCutProbability = spark.udf.register("cutProbability", cutProbability _)

sparkCutProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


cutProbability: (probabilityVector: org.apache.spark.ml.linalg.Vector)Double


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [9]:
val predictionsDF = model
    .transform(inputDF)
    .withColumn("negativeTweetProbability", sparkCutProbability(col("probability")))

predictionsDF = [tweet: string, words: array<string> ... 5 more fields]


[tweet: string, words: array<string> ... 5 more fields]

In [10]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7149b701

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|@lauraklaurak duu...|[@lauraklaurak, d...|(1000,[32,36,60,8...|[4.82771618088120...|[0.48277161808812...|       1.0|      0.5172283819118796|
|Just saw archive ...|[just, saw, archi...|(1000,[60,76,82,1...|[4.68736173928864...|[0.46873617392886...|       1.0|      0.5312638260711353|
|After two years o...|[after, two, year...|(1000,[77,98,128,...|[5.50273788333825...|[0.55027378833382...|       0.0|      0.4497262116661746|
|i'm so bored. the...|[i'm, so, bored.,...|(1000,[82,84,168,...|[4.77490641151282...|[0.47749064115128...|       1.0|      0.5225093588487179|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|I have very few t...|[i, have, very, f...|(1000,[157,299,32...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|
|Great now a slut ...|[great, now, a, s...|(1000,[38,98,170,...|[4.94479054461085...|[0.49447905446108...|       1.0|      0.5055209455389142|
|@vivek1209    Whn...|[@vivek1209, , , ...|(1000,[158,225,37...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|
|have to get up be...|[have, to, get, u...|(1000,[25,82,128,...|[4.85071256985675...|[0.48507125698567...|       1.0|      0.5149287430143249|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|i really could us...|[i, really, could...|(1000,[213,310,32...|[5.65325532419100...|[0.56532553241910...|       0.0|     0.43467446758089984|
|@ishaka pretty go...|[@ishaka, pretty,...|(1000,[36,122,157...|[4.57991330075236...|[0.45799133007523...|       1.0|      0.5420086699247637|
|i want to go home...|[i, want, to, go,...|(1000,[69,76,77,2...|[5.72404822196956...|[0.57240482219695...|       0.0|      0.4275951778030439|
|@Britneezy whyyyy...|[@britneezy, whyy...|(1000,[157,372,38...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|I just burnt a ca...|[i, just, burnt, ...|(1000,[3,4,19,36,...|[4.78925031497775...|[0.47892503149777...|       1.0|      0.5210749685022247|
|No no, it does no...|[no, no,, it, doe...|(1000,[18,130,346...|[5.37392033582394...|[0.53739203358239...|       0.0|      0.4626079664176057|
|A whole day to st...|[a, whole, day, t...|(1000,[170,388,48...|[4.85071256985675...|[0.48507125698567...|       1.0|      0.5149287430143249|
|Um.....I think I ...|[um.....i, think,...|(1000,[8,86,170,3...|[5.12822651677710...|[0.51282265167771...|       0.0|     0.48717734832228904|

...

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|@Gailporter Shit,...|[@gailporter, shi...|(1000,[228,250,38...|[5.46879662125221...|[0.54687966212522...|       0.0|       0.453120337874778|
|Im a lil bored......|[im, a, lil, bore...|(1000,[56,83,86,1...|[5.72763303334968...|[0.57276330333496...|       0.0|     0.42723669666503106|
|@GABROCKIN4HIM I ...|[@gabrockin4him, ...|(1000,[38,208,240...|[5.19776378501233...|[0.51977637850123...|       0.0|      0.4802236214987663|
|@JoeJLover2009 He...|[@joejlover2009, ...|(1000,[111,170,20...|[4.85071256985675...|[0.48507125698567...|       1.0|      0.5149287430143249|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|Life on Mars canc...|[life, on, mars, ...|(1000,[18,32,81,8...|[4.59491449660129...|[0.45949144966012...|       1.0|      0.5405085503398708|
|Im not in the bes...|[im, not, in, the...|(1000,[18,98,163,...|[4.95325055284520...|[0.49532505528452...|       1.0|      0.5046749447154798|
|@CCArquette I wan...|[@ccarquette, i, ...|(1000,[83,163,173...|[4.88664292594539...|[0.48866429259453...|       1.0|       0.511335707405461|
|@Anne_A4L Whats b...|[@anne_a4l, whats...|(1000,[9,26,279,5...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|Kelis and Nas are...|[kelis, and, nas,...|(1000,[44,63,75,8...|[5.60347383478761...|[0.56034738347876...|       0.0|      0.4396526165212384|
|Library for the n...|[library, for, th...|(1000,[36,138,273...|[5.44512755865321...|[0.54451275586532...|       0.0|      0.4554872441346788|
|@lyricsoul I'm so...|[@lyricsoul, i'm,...|(1000,[0,115,329,...|[5.52432120135643...|[0.55243212013564...|       0.0|      0.4475678798643565|
|I stopped followi...|[i, stopped, foll...|(1000,[48,83,91,1...|[5.65325532419100...|[0.56532553241910...|       0.0|     0.43467446758089984|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|  @moyazb oh honey! |[@moyazb, oh, hon...|(1000,[267,462,62...|[4.76389707754090...|[0.47638970775409...|       1.0|      0.5236102922459093|
|studying for the ...|[studying, for, t...|(1000,[36,418,488...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|
|@urbansmiler I do...|[@urbansmiler, i,...|(1000,[18,161,170...|[4.85071256985675...|[0.48507125698567...|       1.0|      0.5149287430143249|
|im saaaaaaaaaaaaa...|[im, saaaaaaaaaaa...|(1000,[47,131,329...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|negativeTweetProbability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+
|finished sewing f...|[finished, sewing...|(1000,[26,36,76,1...|[4.73365253471824...|[0.47336525347182...|       1.0|      0.5266347465281752|
|@chacesmithphoto ...|[@chacesmithphoto...|(1000,[82,130,299...|[5.59995568350210...|[0.55999556835021...|       0.0|      0.4400044316497894|
|not ready for tom...|[not, ready, for,...|(1000,[18,36,106,...|[4.88499036031127...|[0.48849903603112...|       1.0|      0.5115009639688723|
|@rthomasetc don't...|[@rthomasetc, don...|(1000,[69,128,138...|[4.96243156823518...|[0.49624315682351...|       1.0|      0.5037568431764817|

In [6]:

predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

Name: Compile Error
Message: <console>:32: error: not found: value predictionsDF
       predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
       ^

StackTrace: 